In [ ]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.4 MB/s eta 0:00:00


In [ ]:
import datetime
import random

import pandas as pd
from faker import Faker


def gaussian_clamped(rng: random.Random, mu: float, sigma: float, a: float, b: float) -> float:
    # Fonction pour générer un nombre selon une distribution gaussienne, mais clamped
    # (clamped signifie que le nombre est limité entre a et b)
    # Utilise la méthode Box-Muller pour la distribution gaussienne
    val = rng.gauss(mu, sigma)
    # Retourne la valeur, en s'assurant qu'elle est entre a et b
    return max(a, min(b, val))


def generate_orders(
    orders: int,
    start: str = "2024-01-01",
    end: str = "2025-09-01",
    inventory: pd.DataFrame = None,
    customers: pd.DataFrame = None,
    seed: int = None, # Ajout de l'argument seed
) -> pd.DataFrame:
    """
    Génère des données de commandes et les retourne sous forme de DataFrame pandas.

    Args:
        orders: Nombre de commandes à générer
        start: Date de début (YYYY-MM-DD)
        end: Date de fin (YYYY-MM-DD)
        inventory: DataFrame contenant les données d'inventaire (optionnel, améliore le réalisme)
        customers: DataFrame contenant les données clients (optionnel, améliore le réalisme)
        seed: Graine pour le générateur aléatoire (int, optionnel)

    Returns:
        pandas.DataFrame: DataFrame contenant les données de commandes
    """
    # Initialise le générateur de nombres aléatoires avec la graine si fournie
    random.seed(seed)
    rng = random.Random(seed)


    # Convertit les dates de début et de fin en objets datetime avec fuseau horaire UTC
    start_dt = datetime.datetime.fromisoformat(start).replace(tzinfo=datetime.timezone.utc)
    end_dt = datetime.datetime.fromisoformat(end).replace(tzinfo=datetime.timezone.utc)
    # Calcule la différence en secondes entre les deux dates
    delta_seconds = int((end_dt - start_dt).total_seconds())

    # Charge les listes de produits et de clients si les DataFrames sont fournis
    product_ids = list(range(1000, 1250)) # IDs par défaut si l'inventaire n'est pas fourni
    prices = {} # Dictionnaire pour stocker les prix unitaires
    if inventory is not None and not inventory.empty:
        # Utilise les IDs et les prix de l'inventaire fourni
        product_ids = inventory["product_id"].tolist()
        prices = dict(zip(inventory["product_id"], inventory["unit_price"], strict=True))

    customer_ids = list(range(1, 1001)) # IDs par défaut si les clients ne sont pas fournis
    if customers is not None and not customers.empty:
        # Utilise les IDs des clients fournis
        customer_ids = customers["customer_id"].tolist()

    rows = [] # Liste pour stocker les lignes de données des commandes
    for i in range(1, orders + 1):
        # Choisit un ID produit aléatoire
        pid = rng.choice(product_ids)
        # Choisit une quantité aléatoire avec des poids spécifiques (plus de petites quantités)
        qty = rng.choices([1, 2, 3, 4, 5], weights=[0.6, 0.2, 0.12, 0.06, 0.02])[0]
        # Génère une date et heure de vente aléatoire entre les dates de début et de fin
        sold_at = start_dt + datetime.timedelta(seconds=rng.randint(0, delta_seconds))
        # Choisit un ID client aléatoire
        cid = rng.choice(customer_ids)
        # Récupère le prix unitaire de l'inventaire s'il est disponible, sinon génère un prix aléatoire
        unit_price = prices.get(pid, round(rng.uniform(5, 500), 2))
        # Ajoute une ligne à la liste des commandes
        rows.append(
            {
                "id": i, # ID de la commande
                "product_id": pid, # ID du produit
                "customer_id": cid, # ID du client
                "quantity": qty, # Quantité commandée
                "unit_price": unit_price, # Prix unitaire du produit
                "sold_at": sold_at, # Date et heure de la vente
            }
        )

    # Crée un DataFrame pandas à partir de la liste de lignes
    df = pd.DataFrame(rows)

    # S'assure que la colonne "sold_at" est au format datetime
    df["sold_at"] = pd.to_datetime(df["sold_at"])

    # Affiche un message de confirmation
    print(f"✅ Generated {len(df)} orders as DataFrame")
    # Retourne le DataFrame des commandes
    return df


# Listes de catégories et d'adjectifs pour générer des noms de produits
CATEGORIES = [
    ("Apparel", ["T-Shirt", "Hoodie", "Jeans", "Sneakers", "Jacket"]),
    ("Electronics", ["Headphones", "Smartphone", "Tablet", "Smartwatch", "Charger"]),
    ("Home & Kitchen", ["Mug", "Kettle", "Blender", "Vacuum", "Toaster"]),
    ("Beauty", ["Shampoo", "Conditioner", "Face Cream", "Perfume", "Lipstick"]),
    ("Grocery", ["Coffee Beans", "Olive Oil", "Pasta", "Granola", "Tea"]),
]

ADJECTIVES = ["Classic", "Premium", "Eco", "Urban", "Sport", "Comfort", "Pro", "Lite", "Max", "Essential"]


def generate_inventory_data(products: int, seed: int = None) -> pd.DataFrame: # Ajout de l'argument seed
    """
    Génère des données d'inventaire et les retourne sous forme de DataFrame pandas.

    Args:
        products: Nombre de produits à générer
        seed: Graine pour le générateur aléatoire (int, optionnel)

    Returns:
        pandas.DataFrame: DataFrame contenant les données d'inventaire
    """
    # Initialise le générateur de nombres aléatoires avec la graine si fournie
    random.seed(seed)
    rng = random.Random(seed)


    rows = [] # Liste pour stocker les lignes de données de l'inventaire
    product_ids = list(range(1000, 1000 + products)) # Génère une liste d'IDs produits
    for pid in product_ids:
        # Choisit une catégorie et une base de nom aléatoires
        cat, names = rng.choice(CATEGORIES)
        base = rng.choice(names)
        # Choisit un adjectif aléatoire et crée le nom complet du produit
        adj = rng.choice(ADJECTIVES)
        product_name = f"{adj} {base}"
        # Détermine le prix de base selon la catégorie avec une certaine variance
        base_price = {"Apparel": 39, "Electronics": 299, "Home & Kitchen": 79, "Beauty": 25, "Grocery": 12}[cat]
        # Calcule le prix unitaire en utilisant la distribution gaussienne clamped
        price = round(gaussian_clamped(rng, base_price, base_price * 0.25, base_price * 0.4, base_price * 1.8), 2)
        # Calcule la quantité en stock en utilisant la distribution gaussienne clamped (biaisée pour une longue traîne)
        stock_qty = int(gaussian_clamped(rng, 80, 60, 0, 400))
        # Ajoute une ligne à la liste de l'inventaire
        rows.append(
            {
                "product_id": pid, # ID du produit
                "product_name": product_name, # Nom du produit
                "category": cat, # Catégorie du produit
                "unit_price": price, # Prix unitaire
                "stock_quantity": stock_qty, # Quantité en stock
            }
        )

    # Crée un DataFrame pandas à partir de la liste de lignes
    df = pd.DataFrame(rows)
    # Affiche un message de confirmation
    print(f"✅ Generated {len(df)} inventory data as DataFrame")
    # Retourne le DataFrame de l'inventaire
    return df


# -- Generate customers --


def generate_customers(customers: int, seed: int = None) -> pd.DataFrame: # Ajout de l'argument seed
    """
    Génère des données clients et les retourne sous forme de DataFrame pandas.

    Args:
        customers: Nombre de clients à générer
        seed: Graine pour le générateur aléatoire (int, optionnel)

    Returns:
        pandas.DataFrame: DataFrame contenant les données clients
    """
    # Initialise le générateur de nombres aléatoires avec la graine si fournie
    random.seed(seed)
    rng = random.Random(seed)
    # Initialise l'objet Faker pour générer des données réalistes (noms, emails, villes)
    fake = Faker()
    Faker.seed(seed) # Utilise la graine pour Faker

    rows = [] # Liste pour stocker les lignes de données des clients
    channels = [("online", 0.65), ("store", 0.35)] # Liste des canaux avec leurs poids

    for cid in range(1, customers + 1):
        # Génère un nom, un email et une ville réalistes en utilisant Faker
        name = fake.name()
        email = fake.email()
        city = fake.city()
        # Choisit un canal aléatoire en fonction des poids définis
        channel = rng.choices([c for c, _ in channels], weights=[w for _, w in channels])[0]
        # Ajoute une ligne à la liste des clients
        rows.append(
            {
                "customer_id": cid, # ID du client
                "name": name, # Nom du client
                "email": email, # Email du client
                "city": city, # Ville du client
                "channel": channel, # Canal d'acquisition du client
            }
        )

    # Crée un DataFrame pandas à partir de la liste de lignes
    df = pd.DataFrame(rows)
    # Affiche un message de confirmation
    print(f"✅ Generated {len(df)} customers as DataFrame")
    # Retourne le DataFrame des clients
    return df

# Example usage with a fixed seed
seed_value = 11

# Génère les données en utilisant la graine fixe
customers_df = generate_customers(customers=100, seed=seed_value)
inventory_df = generate_inventory_data(products=100, seed=seed_value)
orders_df = generate_orders(orders=100, inventory=inventory_df, customers=customers_df, seed=seed_value)

# Afficher les premières lignes des DataFrames générés
print("\nCustomers DataFrame:")
display(customers_df.head())

print("\nInventory DataFrame:")
display(inventory_df.head())

print("\nOrders DataFrame:")
display(orders_df.head())

✅ Generated 100 customers as DataFrame
✅ Generated 100 inventory data as DataFrame
✅ Generated 100 orders as DataFrame

Customers DataFrame:


,customer_id,name,email,city,channel
0,1,Justin Glass,lhayes@example.org,Port William,online
1,2,Rebecca Davis,donald80@example.net,Oconnellburgh,online
2,3,Sandra Brown,leejohn@example.org,Nguyenshire,store
3,4,Kelly Bowman,fsmith@example.com,Richardside,online
4,5,Sara Zimmerman MD,williamjohnson@example.net,South Danielport,online



Inventory DataFrame:


,product_id,product_name,category,unit_price,stock_quantity
0,1000,Lite Lipstick,Beauty,13.27,115
1,1001,Max Smartphone,Electronics,197.06,92
2,1002,Lite Headphones,Electronics,288.27,104
3,1003,Pro Jacket,Apparel,52.86,60
4,1004,Essential Olive Oil,Grocery,15.66,86



Orders DataFrame:


,id,product_id,customer_id,quantity,unit_price,sold_at
0,1,1057,60,3,9.67,2025-08-29 18:57:36+00:00
1,2,1075,61,1,7.97,2025-02-01 14:40:07+00:00
2,3,1023,19,1,228.24,2024-08-23 15:48:34+00:00
3,4,1088,51,2,39.84,2025-04-07 11:44:27+00:00
4,5,1083,21,2,37.70,2025-05-19 22:12:04+00:00


In [ ]:
# Sauvegarder les DataFrames en fichiers CSV
customers_df.to_csv('customers.csv', index=False)
inventory_df.to_csv('inventory.csv', index=False)
orders_df.to_csv('orders.csv', index=False)

print("✅ DataFrames saved to CSV files: customers.csv, inventory.csv, orders.csv")

✅ DataFrames saved to CSV files: customers.csv, inventory.csv, orders.csv


In [ ]:
from google.colab import files

print("Cliquez sur les liens ci-dessous pour télécharger les fichiers :")
files.download('customers.csv')


Cliquez sur les liens ci-dessous pour télécharger les fichiers :


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('inventory.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('orders.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>